<a href="https://colab.research.google.com/github/ank090/hate_speech_detection/blob/main/hate_speech_detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importing libraries

In [19]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading dataset

In [20]:
dataset=pd.read_csv('final_dataset_basicmlmodel.csv')
tweets=dataset.iloc[:,2].values
dataset

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
5237,31935,1,lady banned from kentucky mall. @user #jcpenn...
5238,31947,1,@user omfg i'm offended! i'm a mailbox and i'...
5239,31948,1,@user @user you don't have the balls to hashta...
5240,31949,1,"makes you ask yourself, who am i? then am i a..."


Cleaning text

In [21]:
def clean(text):
  STOPWORDS.update({'is','the','of','to','on','so'})
  clean_tweet=[]
  for i in range(5242):
    sample=re.sub(r'[^a-zA-Z]'," ",text[i])
    sample=re.sub(r"[.,'\"@#!?]"," ",sample)
    sample=sample.lower()
    sample=sample.split()
    lm=WordNetLemmatizer()
    for word in sample:
      word=lm.lemmatize(word)
    sample=list(set(sample)-STOPWORDS)
    sample=" ".join(sample)
    clean_tweet.append(sample)
  
  return clean_tweet
clean(tweets)

['drags user dysfunctional dysfunction father selfish run kids',
 'use user pdx t lyft credit offer vans wheelchair don thanks getthanked cause disapointed',
 'majesty bihday',
 'take model love ur time u',
 'society factsguide now motivation',
 'talking chaos leave huge allshowandnogo fan fare big pay disputes',
 'danny user camping tomorrow',
 'next school t year revolutionschool exams hate imagine girl think actorslife',
 'cavs love allin won clevelandcavaliers champions cleveland land',
 'user s m gr welcome',
 'blog price may index gold consumer silver forex climbed mom previous ireland',
 'love values pulseshooting biggerproblems orlando heabreaking selfish orlandoshooting standwithorlando',
 'today daddy see gettingfed days',
 'got yugyoem angry junior ouch omg',
 'positive paner thankful',
 'user smiles ig make friday cookies around people via',
 'know essential made chemicals oils',
 'conceded knowing ha gave people blaming hit free away kick euro fat goal rooney bale',
 'litt

Creating tokens

In [22]:
cv=CountVectorizer(max_features=10000)
x=cv.fit_transform(clean(tweets)).toarray()
y=dataset.iloc[:,1].values

Splitting dataset in to test and training set

In [23]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=0)

Training model on training set

In [24]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50,criterion='entropy')#,max_features=10000)
rf.fit(xtrain,ytrain)
ypred=rf.predict(xtest)

Checking accuracy

In [25]:
from sklearn.metrics import classification_report
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.82      0.92      0.86       887
           1       0.87      0.73      0.80       686

    accuracy                           0.84      1573
   macro avg       0.84      0.83      0.83      1573
weighted avg       0.84      0.84      0.83      1573

